In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

In [2]:
df_test = pd.read_csv("./final_test.csv")

In [3]:
def cal_revenue(df):
    no_stay = []
    no_people = []
    for i in range(len(df)):
        if df["stays_in_week_nights"].iloc[i] == 0 and df["stays_in_weekend_nights"].iloc[i] == 0:
            no_stay.append(i)
        if df["adults"].iloc[i] == 0 and df["children"].iloc[i] == 0 and df["babies"].iloc[i] == 0:
            no_people.append(i)
    index_drop = np.unique(no_stay + no_people)
#     print("刪去%d行" %len(index_drop))
    df = df.drop(index_drop, axis=0)

    df = df[df["is_canceled"] == 0]
    df = df[df["adr"] > 0]
    
    adr = df["adr"].values
    
    stay_days = df["stays_in_week_nights"] + df["stays_in_weekend_nights"]
    stay_days = stay_days.values
    
    # 生成月份對應數字的字典
    month_name = df["arrival_date_month"].unique()
    tmp = [7, 8, 9 ,10, 11, 12, 1, 2, 3, 4, 5, 6]
    month_num = {}
    for i in range(len(month_name)):
        month_num[month_name[i]] = tmp[i]

    # 生成所有日期
    num = 0
    all_date = []
    keys = []
    for i in range(len(df)):
        year = df["arrival_date_year"].iloc[i]
        month = month_num[df["arrival_date_month"].iloc[i]]
        day = df["arrival_date_day_of_month"].iloc[i]
        date = "%d-%02d-%02d" %(year, month, day)
        keys.append(date)
        if date not in all_date:
                all_date.append(date)
                
    all_adr = []
    all_day = []
    for i in range((len(all_date))):
        adr_day = 0
        for j in range(len(keys)):
            if keys[j] == all_date[i]:
                adr_day = adr_day + (adr[j]*stay_days[j])
        all_adr.append(adr_day)
    all_adr = np.array(all_adr)
    
    all_revenue = []
    for i in range(len(all_adr)):
        revenue = all_adr[i] / 10000
        revenue = int(revenue)
        if revenue >= 9:
            revenue = 9
        if revenue <= 0:
            revenue = 0
        all_revenue.append(revenue)
    all_revenue = np.array(all_revenue)
    return all_revenue

In [4]:
revenue = cal_revenue(df_test)
print(revenue.shape)
print(revenue)

(153,)
[2 2 4 2 3 2 2 4 4 3 3 2 4 4 3 3 4 2 2 2 3 3 3 3 1 1 2 5 5 3 4 4 3 3 4 4 3
 4 3 3 4 3 3 4 3 3 3 4 3 3 4 4 2 3 5 3 2 4 3 2 2 4 5 5 4 5 3 2 4 4 4 6 4 3
 3 4 3 5 4 4 4 4 3 3 4 4 4 3 3 3 4 6 4 6 5 3 4 4 5 6 6 6 4 5 5 8 6 8 5 3 4
 4 6 6 8 5 5 5 5 8 5 6 6 6 7 6 6 7 9 4 5 5 4 8 6 6 5 6 7 6 6 7 6 4 5 5 5 7
 5 8 3 3 4]


In [5]:
test_label = pd.read_csv("./test_nolabel.csv")
new_pred = test_label.copy()
new_pred.insert(loc=1, column="label", value=revenue)
new_pred.to_csv("/home/data/andy/course/ML_Lin/new_pred.csv", index=0)